# Fit Cox Models

## Initialize

In [2]:
import os
import math
import pathlib
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from IPython.display import clear_output
import ray
import datetime
import subprocess
import warnings
import lifelines
from lifelines.utils import CensoringType
from lifelines.utils import concordance_index

In [3]:
base_path = "/home/jakobs"

project_path = f"{base_path}/data"

#experiment = '230629'
#experiment_path = f"{project_path}/{experiment}"
#pathlib.Path(experiment_path).mkdir(parents=True, exist_ok=True)

partitions = [i for i in range(10)]

In [4]:
os.environ['MKL_NUM_THREADS'] = "1"
os.environ['NUMEXPR_NUM_THREADS'] = "1"
os.environ['OMP_NUM_THREADS'] = "1"

In [5]:
from scripts.coxph_fit_partition import load_data, fit_endpoint

In [6]:
partitions

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [7]:
for partition in tqdm(partitions):
    eids_dict, score_defs, endpoint_defs, endpoints, models, model_path, experiment_path, data_partition = load_data(partition)
    # setup ray and put files in plasma storage
    #ray.init(num_cpus=24) # crashes if num_cpus > 16, why not more possible?
    ray_eids = ray.put(eids_dict)
    ray_score_defs = ray.put(score_defs)
    ray_endpoint_defs = ray.put(endpoint_defs)
    ray_partition = ray.put(data_partition)
    # fit cox models via ray
    progress = []
    for endpoint in endpoints:
        progress.append(fit_endpoint.remote(ray_partition, ray_eids, ray_score_defs, ray_endpoint_defs, endpoint, partition, models, model_path, experiment_path))
    [ray.get(s) for s in tqdm(progress)]

  0%|          | 0/10 [00:00<?, ?it/s]

2023-11-18 17:28:15,628	INFO worker.py:1544 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


  0%|          | 0/367 [00:00<?, ?it/s]

  0%|          | 0/367 [00:00<?, ?it/s]

  0%|          | 0/367 [00:00<?, ?it/s]

...

  0%|          | 0/367 [00:00<?, ?it/s]

  0%|          | 0/367 [00:00<?, ?it/s]

  0%|          | 0/367 [00:00<?, ?it/s]

  0%|          | 0/367 [00:00<?, ?it/s]

  0%|          | 0/367 [00:00<?, ?it/s]

## Check progress

In [8]:
cox_paths = os.listdir(f"{experiment_path}/coxph/models/")
#len(cox_paths), cox_paths[0]
path_df = pd.DataFrame(data = [p[:-2] for p in cox_paths]).rename(columns={0:"path"})
print(path_df.head())
path_df[["endpoint_1", "endpoint_2", "score", "model", "partition"]] = path_df["path"].str.split("_", expand=True)

                                    path
0       phecode_168-1_SCORE2_RetinaUKB_9
1  phecode_582_SCORE2+Retina_RetinaUKB_6
2       phecode_714-3_SCORE2_RetinaUKB_0
3     phecode_431-12_Age+Sex_RetinaUKB_1
4          phecode_403_ASCVD_RetinaUKB_9


In [9]:
experiment_path

'/home/jakobs/data/231117'

In [10]:
experiment_path

'/home/jakobs/data/231117'

In [11]:
cox_paths

['phecode_168-1_SCORE2_RetinaUKB_9.p',
 'phecode_582_SCORE2+Retina_RetinaUKB_6.p',
 'phecode_714-3_SCORE2_RetinaUKB_0.p',
 'phecode_431-12_Age+Sex_RetinaUKB_1.p',
 'phecode_403_ASCVD_RetinaUKB_9.p',
 'phecode_420_ASCVD+Retina_RetinaUKB_5.p',
 'phecode_444-1_ASCVD_RetinaUKB_1.p',
 'phecode_413-1_SCORE2_RetinaUKB_7.p',
 'phecode_326_Age+Sex+Retina_RetinaUKB_1.p',
 'phecode_139-6_Age+Sex_RetinaUKB_5.p',
 'phecode_324_ASCVD_RetinaUKB_4.p',
 'phecode_522-11_ASCVD+Retina_RetinaUKB_2.p',
 'phecode_440-1_Retina_RetinaUKB_2.p',
 'phecode_585_SCORE2+Retina_RetinaUKB_1.p',
 'phecode_120-2_ASCVD+Retina_RetinaUKB_1.p',
 'phecode_469_Age+Sex_RetinaUKB_8.p',
 'phecode_144-2_Age+Sex+Retina_RetinaUKB_3.p',
 'phecode_522_Age+Sex+Retina_RetinaUKB_1.p',
 'phecode_722_ASCVD+Retina_RetinaUKB_1.p',
 'phecode_554-1_ASCVD+Retina_RetinaUKB_9.p',
 'phecode_401-3_Age+Sex+Retina_RetinaUKB_5.p',
 'phecode_106-3_Retina_RetinaUKB_4.p',
 'phecode_092-2_ASCVD+Retina_RetinaUKB_4.p',
 'phecode_529-2_Age+Sex+Retina_Retina

In [12]:
path_df["endpoint"] = path_df["endpoint_1"] + "_" + path_df["endpoint_2"] 

path_df.value_counts(["partition"]).to_frame()

,0
partition,
0,2569
1,2569
2,2569
3,2569
4,2569
5,2569
6,2569
7,2569
8,2569
